<a href="https://colab.research.google.com/github/jackychencw/MIE1624_Course_Project_Group19/blob/Lawrence-nlp-preprocess/MIE1624_proj_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Neural Network**

In [11]:
import pandas as pd
import tensorflow as tf
import pickle

from tensorflow.keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Reshape
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.optimizers.schedules import ExponentialDecay
from keras.optimizers import Adam

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.svm import SVC, LinearSVR
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error as mse
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.compose import ColumnTransformer
from sklearn.base import TransformerMixin

from google.colab import drive

drive.mount('/content/drive')

data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/jacky_train_data.csv", index_col=0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
data

,overall,reviewTime,reviewerID,reviewText,summary,unixReviewTime,price,itemID,reviewHash,important_features,important_features(clean),category_Alternative Rock,category_Classical,category_Dance & Electronic,category_Jazz,category_Pop
0,5.0,"08 26, 2014",u92990698,A contemporary jazz and soul performer who's m...,""" CLASSIQUE """,1409011200,8.40,p23649501,3856620,u92990698 p23649501 A contemporary jazz and so...,u92990698 p23649501 contemporary jazz soul per...,0,0,0,1,0
1,5.0,"05 11, 2003",u36200649,Very good idea to put both the 'pop' and 'orch...,Exceeded my Expectations - This album RAWKS!,1052611200,10.98,p58458313,56086781,u36200649 p58458313 Very good idea to put both...,u36200649 p58458313 good idea put pop orchestr...,1,0,0,0,0
2,5.0,"12 5, 2017",u10721702,This is a great collection of Carole King's so...,A Must-have for Carole King Fans,1512432000,5.99,p97027626,55852154,u10721702 p97027626 This is a great collection...,u10721702 p97027626 great collection carole ki...,0,0,0,0,1
3,5.0,"09 24, 2013",u86003775,The is album is a brilliant piece of Jazz fusi...,A Master piece!,1379980800,14.64,p43167086,43228100,u86003775 p43167086 The is album is a brillian...,u86003775 p43167086 album brilliant piece jazz...,0,0,0,1,0
4,5.0,"01 14, 2015",u25672859,Waited a LONG time for this DVD to be released...,especially if you like concert videos,1421193600,9.92,p94494236,54425467,u25672859 p94494236 Waited a LONG time for thi...,u25672859 p94494236 waited long time dvd relea...,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,5.0,"09 12, 2013",u29136602,There is just something about Wesley's voice t...,Just right.,1378944000,9.49,p26326919,90575108,u29136602 p26326919 There is just something ab...,u29136602 p26326919 something wesley voice mak...,1,0,0,0,0
149996,5.0,"05 15, 2016",u51715193,How many recordings of Verdis Requiem do you o...,Being there in vibrant immediacy of performan...,1463270400,14.83,p69247882,74425587,u51715193 p69247882 How many recordings of Ver...,u51715193 p69247882 many recordings verdis req...,0,1,0,0,0
149997,5.0,"06 13, 2004",u28696060,....their best overall album. After the revis...,Creative rebirth and possibly....,1087084800,18.86,p07028930,20399056,u28696060 p07028930 ....their best overall alb...,u28696060 p07028930 best overall album revisio...,1,0,0,0,0
149998,5.0,"06 27, 2012",u04925906,"Believe it or not, once upon a time, artists l...",Once Upon A Time,1340755200,19.99,p38606785,5931588,"u04925906 p38606785 Believe it or not, once up...",u04925906 p38606785 believe not upon time arti...,0,0,0,0,1


In [3]:
data.drop(columns=[
                   'reviewerID', 'reviewText', 'unixReviewTime', 'itemID', 'reviewHash', 'important_features'
], inplace=True)

In [4]:
data['reviewTime'] = data.reviewTime.apply(lambda _: int(_[-4:]))
dummy_years = pd.get_dummies(data.reviewTime, prefix='year')
data = pd.concat([data, dummy_years], axis=1)
data.drop(columns=['reviewTime'], inplace=True)

In [5]:
saved_features = 2000


y = data.overall
X = data.drop(columns=['overall'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ncolumn = X.shape[1] + 2 * saved_features - 2

In [6]:
test_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/jacky_test_data.csv", index_col=0)
test_data

,reviewTime,reviewerID,reviewText,summary,unixReviewTime,price,itemID,reviewHash,important_features,category_Alternative Rock,category_Classical,category_Dance & Electronic,category_Jazz,category_Pop,important_features(clean)
0,"02 11, 2017",u04140621,ok,Five Stars,1486771200,31.20,p65721979,98944233,u04140621 p65721979 ok,0,0,0,0,1,u04140621 p65721979 ok
1,"01 29, 2018",u74766187,The 1955 and 1958 sets alone are worth the pur...,Buy It For the 1955 Set With Monk!,1517184000,34.04,p87809670,33903910,u74766187 p87809670 The 1955 and 1958 sets alo...,0,0,0,1,0,u74766187 p87809670 1955 1958 sets alone worth...
2,"08 29, 2005",u31689638,Mary J Blige is someone most people know of. S...,Mary's excellent debut! Classic!,1125273600,1.72,p52702240,35386470,u31689638 p52702240 Mary J Blige is someone mo...,0,0,0,0,1,u31689638 p52702240 mary j blige someone peopl...
3,"11 2, 2015",u35613516,Very good satch album,Four Stars,1446422400,10.99,p01314374,12404766,u35613516 p01314374 Very good satch album,0,0,0,0,1,u35613516 p01314374 good satch album
4,"01 3, 2012",u93320378,Well there are hundreds of Christmas anthologi...,CD lives up to its name,1325548800,7.98,p84396269,68253688,u93320378 p84396269 Well there are hundreds of...,1,0,0,0,0,u93320378 p84396269 well hundreds christmas an...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,"10 8, 2012",u16765812,"My goodness, Christian Scott has simply taken ...",Blown Away,1349654400,11.87,p17291597,66654690,"u16765812 p17291597 My goodness, Christian Sco...",0,0,0,1,0,u16765812 p17291597 goodness christian scott s...
19996,"10 26, 2012",u04428712,Congratulations Clan of Xymox - You have butch...,Take Shelter! These covers are abysmal!,1351209600,16.99,p73630774,55746555,u04428712 p73630774 Congratulations Clan of Xy...,0,0,0,0,1,u04428712 p73630774 congratulations clan xymox...
19997,"10 22, 2005",u39251384,I'm so happy that all of Cher's videos are out...,So glad it's here!,1129939200,49.95,p58275451,78558928,u39251384 p58275451 I'm so happy that all of C...,0,0,0,0,1,u39251384 p58275451 happy cher videos dvd reme...
19998,"08 30, 2005",u05089888,While I agree with the comments made by Maxwel...,A personal must-have,1125360000,9.42,p13155723,84815536,u05089888 p13155723 While I agree with the com...,0,1,0,0,0,u05089888 p13155723 agree comments made maxwel...


In [7]:

test_data.drop(columns=[
                   'reviewerID', 'reviewText', 'unixReviewTime', 'itemID', 'reviewHash', 'important_features'
], inplace=True)

test_data['reviewTime'] = test_data.reviewTime.apply(lambda _: int(_[-4:]))
dummy_years = pd.get_dummies(test_data.reviewTime, prefix='year')
test_data = pd.concat([test_data, dummy_years], axis=1)
test_data.drop(columns=['reviewTime'], inplace=True)
test_data = test_data.reindex(columns=X.columns)

In [22]:
class DenseTransformer(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()
        

# def baseline_model(
#     ncolumn = ncolumn,
#     loss_func='mse',
#     opt='Adam',
#     metrics = [tf.keras.losses.MSE]
#     ):
#   model = tf.keras.Sequential([
#     Dense(ncolumn, activation='relu'),
#     Dropout(0.5),
#     Dense(2*ncolumn, activation='relu'),
#     Dropout(0.5),
#     Dense(2*ncolumn, activation='relu'),
#     Dropout(0.5),
#     Dense(2*ncolumn, activation='relu'),
#     Dropout(0.5),
#     Dense(2*ncolumn, activation='relu'),
#     Dropout(0.5),
#     Dense(1, kernel_initializer='normal', activation='linear')
#     ])
#   model.compile(loss=loss_func, optimizer=opt, metrics=metrics)
#   return model

def baseline_model(
    ncolumn = ncolumn,
    loss_func='mse',
    # opt='Adam',
    metrics = [tf.keras.losses.MSE]
    ):
  model = tf.keras.Sequential([
    Reshape(target_shape=(1,-1)),
    LSTM(128),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2*128, activation='relu'),
    Dropout(0.5),
    Dense(1, kernel_initializer='normal', activation='linear')
  ])
  lr_schedule = ExponentialDecay(
      initial_learning_rate=0.005,
      decay_steps=3000,
      decay_rate=0.95,
      staircase=True)

  opt = Adam(learning_rate=lr_schedule)
  model.compile(loss=loss_func, optimizer=opt, metrics=metrics)
  return model


vectorizers = {
    # 'wf': CountVectorizer(),
    'tfidf': TfidfVectorizer(),
    # '2gram': CountVectorizer(analyzer='word', ngram_range=(2, 2)),
        
    }

# es = EarlyStopping(monitor='mean_squared_error', mode='min', patience=3)
checkpoint_filepath = "/content/drive/MyDrive/Colab Notebooks/checkpoints"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='mean_squared_error',
    mode='min',
    save_best_only=True)



# for batch_size in [100, 10000, 1000]:
# for batch_size in [64]:
#   for name1, vc1 in vectorizers.items():
#       summary_pipe = Pipeline([
#                                 ('vect', vc1),
#                                 ('select', SelectKBest(chi2, k=saved_features))
#                                 ])

#       review_pipe = Pipeline([('vect', TfidfVectorizer()),
#                                     ('select', SelectKBest(chi2, k=saved_features))
#                                     ])

#       preprocess = ColumnTransformer([
#                                         ('price_std', StandardScaler(), ['price']),
#                                         ('summary_count_vec', summary_pipe, 'summary'),
#                                         ('review_tfidf', review_pipe, 'important_features(clean)')
#                                         ], remainder = 'passthrough')




#       model = KerasRegressor(baseline_model,epochs=50, batch_size=batch_size, verbose=True)
#       pipe = Pipeline([('preprocess', preprocess),
#                         ('to_dense', DenseTransformer()),
#                         ('nn', model)])

#       # pipe.fit(X_train, y_train)
#       # pred = pipe.predict(X_test)
#       # error = mse(pred, y_test)
#       # print(f"vc1_{name1} vc2_tfidf neural network, test MSE: {error}")

#       # prediction = pipe.predict(test_data)
#       # pd.DataFrame(prediction).to_csv(f"/content/drive/MyDrive/Colab Notebooks/data/vc1{name1}_vc2tfidf_{saved_features}_features_batch_{batch_size}_epoch_50_prediction.csv")

#       # kfold = KFold(n_splits=10)
#       kfold = KFold(n_splits=5)
#       results = cross_val_score(pipe, X_train, y_train, cv=kfold, n_jobs=2)
#       error = results.mean()
#       print("Baseline: %.2f (%.2f) MSE" % (error, results.std()))

#       if results.mean() < 0.5:
#         prediction = pipe.predict(test_data)
#         pd.DataFrame(prediction).to_csv(f"/content/drive/MyDrive/Colab Notebooks/data/test_error_{error}.csv")

In [23]:
summary_pipe = Pipeline([
                                ('vect', vc1),
                                ('select', SelectKBest(chi2, k=saved_features))
                                ])

review_pipe = Pipeline([('vect', TfidfVectorizer()),
                              ('select', SelectKBest(chi2, k=saved_features))
                              ])

preprocess = ColumnTransformer([
                                  ('price_std', StandardScaler(), ['price']),
                                  ('summary_count_vec', summary_pipe, 'summary'),
                                  ('review_tfidf', review_pipe, 'important_features(clean)')
                                  ], remainder = 'passthrough')




model = KerasRegressor(baseline_model,epochs=50, batch_size=batch_size, verbose=True)
pipe = Pipeline([('preprocess', preprocess),
                  ('to_dense', DenseTransformer()),
                  ('nn', model)])
pipe.fit(X_train, y_train)

Epoch 1/50
1875/1875 [==============================] - 7s 3ms/step - loss: 1.2969 - mean_squared_error: 1.2969
Epoch 2/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5181 - mean_squared_error: 0.5181
Epoch 3/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4404 - mean_squared_error: 0.4404
Epoch 4/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4124 - mean_squared_error: 0.4124
Epoch 5/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3913 - mean_squared_error: 0.3913
Epoch 6/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3674 - mean_squared_error: 0.3674
Epoch 7/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3501 - mean_squared_error: 0.3501
Epoch 8/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3313 - mean_squared_error: 0.3313
Epoch 9/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3184 - mean_squared_error:

Pipeline(memory=None,
         steps=[('preprocess',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('price_std',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['price']),
                                                 ('summary_count_vec',
                                                  Pipeline(memory=None,
                                                           steps=[('vect',
                                                                   TfidfVectorizer(analyzer='word',
                                                                           

In [24]:
def grid_test(model, X_test, y_test):
  y_pred = model.predict(X_test)
  score = mean_squared_error(y_test, y_pred)
  return score, y_pred

In [25]:
train_scores, train_predictions = grid_test(pipe, X_train, y_train)
scores, predictions = grid_test(pipe, X_test, y_test)
print(train_scores)
print(scores)

469/469 [==============================] - 2s 2ms/step
0.09935362936149413
0.4701895012101594


In [29]:
pipe.fit(X, y)
predictions = pipe.predict(test_data)

Epoch 1/50
2344/2344 [==============================] - 10s 4ms/step - loss: 1.1924 - mean_squared_error: 1.1924
Epoch 2/50
2344/2344 [==============================] - 9s 4ms/step - loss: 0.4989 - mean_squared_error: 0.4989
Epoch 3/50
2344/2344 [==============================] - 9s 4ms/step - loss: 0.4457 - mean_squared_error: 0.4457
Epoch 4/50
2344/2344 [==============================] - 9s 4ms/step - loss: 0.4114 - mean_squared_error: 0.4114
Epoch 5/50
2344/2344 [==============================] - 9s 4ms/step - loss: 0.3823 - mean_squared_error: 0.3823
Epoch 6/50
2344/2344 [==============================] - 9s 4ms/step - loss: 0.3617 - mean_squared_error: 0.3617
Epoch 7/50
2344/2344 [==============================] - 9s 4ms/step - loss: 0.3480 - mean_squared_error: 0.3480
Epoch 8/50
2344/2344 [==============================] - 9s 4ms/step - loss: 0.3303 - mean_squared_error: 0.3303
Epoch 9/50
2344/2344 [==============================] - 9s 4ms/step - loss: 0.3147 - mean_squared_error

In [31]:
rating_pairs_path = "/content/drive/MyDrive/Colab Notebooks/data/rating_pairs.csv"
def export_to_kaggle(rating_pairs_path, predictions):
  rating_pairs = pd.read_csv(rating_pairs_path)
  rating_pairs['prediction'] = predictions
  rating_pairs.to_csv("/content/drive/MyDrive/Colab Notebooks/data/rating_pairs_pred.csv", index=False)

In [32]:
export_to_kaggle(rating_pairs_path, predictions)

In [ ]:
pred = pipe.predict(X_test)
error = mse(pred, y_test)
print(f"neural network, test MSE: {error}")

prediction = pipe.predict(test_data)
pd.DataFrame(prediction).to_csv(f"/content/drive/MyDrive/Colab Notebooks/data/tfidf_{saved_features}_features_batch_{batch_size}_epoch_200_prediction.csv")